### Hanoi mit Klassen
Vergleiche mit dem Notebook `Tuerme_von_Hanoi.ipynb` der Lektion 12.

### Die Klasse `Game`
Diese Klasse ist zust&auml;ndig f&uuml;r die Spiellogik, das verwalten und modifiziern der Scheibenstapel.

Beachte: `event_handler` ist ein Attribut der Instanz. Dieses Attribut speichert eine Funktion, die
von den Methoden `new_game` und `move_disk` aufgerufen wird. 
Diese Funktion soll den aktuellen Spielzustand darstellen, hier die 3 Stapel. 

Der Default Eventhandler ist die Funktion `print`.
Wir geben einfach die Argumente aus, mit denen die Funktion `event_handler` aufgerufen wird.

In [ ]:
class Game:
    def __init__(self):
        self.ndisks = 4
        self.stacks = None
        self.event_handler = print

    def new_game(self):
        self.stacks = [list(range(self.ndisks))[::-1], [], []]
        self.event_handler('new_game', self.stacks)

    def move_disk(self, src, dst):
        if (not self.stacks[src] or
           (self.stacks[dst] and self.stacks[dst][-1] < self.stacks[src][-1])):
            return
        disk = self.stacks[src].pop()
        self.stacks[dst].append(disk)
        self.event_handler('update_stacks', self.stacks)

    def __repr__(self):
        return str(self.stacks)

In [ ]:
game = Game()
game

In [ ]:
game.new_game()

In [ ]:
game.move_disk(0, 2)

In [ ]:
game

### Die Klasse View
Die Klasse `View` ist zust&auml;ndig f&uuml;r die graphische Darstellung der
Scheibenstapel. Wir &uuml;bergeben der `__init__` Methode eine Game-Instanz, damit dem
Attribut `event_hander` der Game-Instanz die Methode `callback` der View zugewiesen werden kann.
Die Funktion `callback` ignoriert das Event und zeichnet den Scheibenstapel.

Die View-Instanz hat ein `canvas` Attribut, welches die Leinwand speichert auf die gezeichnet wird.

Die Methode `_ipython_display_` wird aufgerufen, wenn eine Instanz der `View` dargestellt werden soll.
Diese Methode zeigt einfach die Leinwand an.

In [ ]:
import widgets_helpers
from ipycanvas import Canvas
from IPython.display import display


class CanvasView:
    positions = [50, 150, 250]
    disk_widths = [30, 50, 70, 90]
    disk_height = 10
    colors = ['brown', 'teal', 'blue', 'purple']

    def __init__(self, game):
        game.event_handler = self.callback
        self.canvas = widgets_helpers.new_canvas()

    def draw_stack(self, stack, pos):
        h = self.disk_height
        for i, disk in enumerate(stack):
            self.canvas.fill_style = self.colors[disk]
            w = self.disk_widths[disk]
            self.canvas.fill_rect(pos - w/2,
                                  self.canvas.height-(i+1)*h,
                                  w,
                                  h)

    def draw_stacks(self, stacks):
        self.canvas.clear()
        for pos, stack in zip(self.positions, stacks):
            self.draw_stack(stack, pos)

    def callback(self, event, data):
        self.draw_stacks(data)

    def _ipython_display_(self):
        display(self.canvas)

In [ ]:
view = CanvasView(game)
view

In [ ]:
# Methode draw_stacks testen
stacks = [[2, 1], [3, 2, 1], [1, 2, 3]]
view.draw_stacks(stacks)

In [ ]:
game.new_game()

In [ ]:
game.move_disk(0, 1)

### Die Klasse Controller


In [ ]:
import widgets_helpers
import canvas_callbacks
from IPython.display import display


class Controller:

    err_out = widgets_helpers.new_output()

    def __init__(self, game, canvas):
        self.game = game
        self.canvas = canvas
        self.src = None

        canvas_callbacks.remove_all_callbacks(self.canvas)
        self.canvas.on_key_down(self.on_key_down)

    @err_out.capture()
    def on_key_down(self, key, *flags):
        if key in '123' and self.src is None:
            self.src = int(key) - 1
            return
        elif key in '123':
            dst = int(key) - 1
            self.game.move_disk(self.src, dst)
        elif key == 'n':
            self.game.new_game()
        self.src = None

    def _ipython_display_(self):
        display(self.canvas, self.err_out)

In [ ]:
controller = Controller(game, view.canvas)
controller

In [ ]:
game.new_game()

In [ ]:
game.move_disk(0, 1)

In [ ]:
# Error triggern
controller.on_key_down()

In [ ]:
controller.err_out.clear_output()

### Aufgabe
Erweitere die Klasse `Controller` um Methoden
`on_mouse_down` und `on_mouse_up`, welche das Verschieben der Scheiben mit der Maus 
m&ouml;glich machen sollen. Orientiere dich an der Maussteuerung im Notebook `Tuerme_von_Hanoi.ipynb` der Lektion 12.
Passe dann die `__init__`-Methode entsprechend an, um die neuen Eventhandlers zu registrieren.



```python

    def __init__(self, game, canvas):
        ...
        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)
        self.canvas.on_mouse_up(self.on_mouse_up)
        ...

    @err_out.capture()
    def on_mouse_down(self, x, y):
        ...

    @err_out.capture()
    def on_mouse_up(self, x, y):
        ...
```